# A verbose walk thru of gaussian-process-regression emulation for global optimization

In this notebook, we go thru much pain to demonstrate how GPR can be used for so-called "bayesian optimization" with a very naive in-filling strategy.  Many of the steps and operations in this notebook are done only for the purposes of hoping to explain how GPR emulation works.

We will be using the well-known hosaki function for this analysis

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyemu

In [ ]:
org_d = os.path.join("..","autotest","utils","hosaki_template")
assert os.path.exists(org_d)

In [ ]:
t_d = "hosaki_template"
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_d,t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))

In [ ]:
par = pst.parameter_data
par

The hosaki function only has two decision variables - its very simple.  But in general, GPR-based emulation does not scale to extreme dimensions (like those seem in ensemble-based DA), but well within the space of global (multi-objective) optimization, so its an obvious pairing with pestpp-mou...

The hosaki function only has 1 objective and no constraints and our goal is to minimize this single objective:

In [ ]:
pst.observation_data

First, lets sweep over decision variable space so that we can map the "true" objective function surface:

In [ ]:
pvals = []
sweep_steps = 20
for p1 in np.linspace(par.parlbnd.iloc[0],par.parubnd.iloc[0],sweep_steps):
    for p2 in np.linspace(par.parlbnd.iloc[0],par.parubnd.iloc[0],sweep_steps):
        pvals.append([p1,p2])
pvals = pd.DataFrame(pvals,columns=pst.par_names)
pvals.to_csv(os.path.join(t_d,"sweep_in.csv"))
pst.pestpp_options["ies_par_en"] = "sweep_in.csv"
pst.control_data.noptmax = -1
sweep_d = os.path.join("hosaki_sweep")
pst.pestpp_options["ies_include_base"] = False

pst.write(os.path.join(t_d,"pest.pst"))
port = 5544
num_workers = 50

pyemu.os_utils.start_workers(t_d,"pestpp-ies","pest.pst",
                             num_workers=num_workers,
                             master_dir=sweep_d,worker_root='.',
                             port=port)


In [ ]:
sweep_pe = pd.read_csv(os.path.join(sweep_d,"pest.0.par.csv"),index_col=0)
sweep_oe = pd.read_csv(os.path.join(sweep_d,"pest.0.obs.csv"),index_col=0)
sweep_x = sweep_pe.loc[:,pst.par_names[0]].values.reshape(sweep_steps,sweep_steps)
sweep_y = sweep_pe.loc[:,pst.par_names[1]].values.reshape(sweep_steps,sweep_steps)
sweep_z = sweep_oe.loc[:,pst.obs_names[0]].values.reshape(sweep_steps,sweep_steps)

In [ ]:
def get_obj_map(ax,_sweep_x,_sweep_y,_sweep_z,label="objective function",levels=[-2,-1,0]): 
    cb = ax.pcolormesh(_sweep_x,_sweep_y,_sweep_z)
    plt.colorbar(cb,ax=ax,label=label)
    ax.contour(_sweep_x,_sweep_y,_sweep_z,levels=levels,colors="w")
    ax.set_aspect("equal")
    return ax

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5))
_ = get_obj_map(ax,sweep_x,sweep_y,sweep_z)
ax.set_title("truth",loc="left")
plt.show()
plt.close(fig)

So that is the true objective function surface with the global minimum at (4,2) and a local minimum at (1,2).

The GPR BO workflow starts by first evaluating an initial population with the "model", which here is just the simple hosaki function.  In practice, this will be an expensive and complex process based model.  To make this demo more interesting, we will limit the decision variable search space to not include the global minimum (and really 10 members in the initial population is pretty crazy small - just for demo!)

In [ ]:
m_d = "hosaki_model_master"
if os.path.exists(m_d):
    shutil.rmtree(m_d)
pst.pestpp_options["mou_population_size"] = 10
pst.control_data.noptmax = -1
par = pst.parameter_data
par.loc[pst.par_names[0],"parubnd"] = 3
par.loc[pst.par_names[0],"parval1"] = 1.5
pst.write(os.path.join(t_d,"pest.pst"))
num_workers = 10
pyemu.os_utils.start_workers(t_d,"pestpp-mou","pest.pst",
                                 num_workers=num_workers,
                                 master_dir=m_d,worker_root='.',
                                 port=port)


In [ ]:
training_dvpop_fname = os.path.join(m_d,"pest.0.dv_pop.csv")
training_opop_fname = os.path.join(m_d,"pest.0.obs_pop.csv")

In [ ]:
training_dvpop = pd.read_csv(training_dvpop_fname,index_col=0)
training_opop = pd.read_csv(training_opop_fname,index_col=0)
fig,ax = plt.subplots(1,1,figsize=(6,5))
get_obj_map(ax,sweep_x,sweep_y,sweep_z)
ax.scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
ax.set_title("true objective function surface with training points")
plt.show()
plt.close(fig)
           

Those white triangles are the places where we have actually run the model - in practice we will never have the colormap or the contours - those are just to help us understand what is happening...

This function is where the magic happens: we use the decision variable population and resulting output/observation population to setup the GPR emulation process.  Note that `pyemu.helpers.prep_for_gpr()` function setups and training a GPR emulator for each non-zero weighted observation in the control file... 

In [ ]:
def prep_for_gpr(dvpops,opops,gpr_t_d,noptmax=-1):
    pyemu.helpers.prep_for_gpr(os.path.join(t_d,"pest.pst"),dvpops,opops,gpr_t_d=gpr_t_d,plot_fits=True)
    gpst = pyemu.Pst(os.path.join(gpr_t_d,"pest.pst"))
    #shutil.copy2(dvpops[-1],os.path.join(gpr_t_d,"initial_dv_pop.csv"))
    #gpst.pestpp_options["mou_dv_population_file"] = "initial_dv_pop.csv"
    shutil.copy2(os.path.join(sweep_d,"sweep_in.csv"),os.path.join(gpr_t_d,"sweep_in.csv"))
    gpst.control_data.noptmax = noptmax
    gpst.pestpp_options["ies_include_base"] = False
    gpst.pestpp_options["mou_save_population_every"] = 1
    gpst.pestpp_options.pop("mou_dv_population_file",None)
    par = gpst.parameter_data
    par.loc[:,"parlbnd"] = 0.0
    par.loc[:,"parubnd"] = 5.0
    par.loc[:,"parval1"] = 2.5    
    
    gpst.write(os.path.join(gpr_t_d,"pest.pst"),version=2)
    return gpst

In [ ]:
training_dvpop_fnames = [training_dvpop_fname]
training_opop_fnames = [training_opop_fname]

In [ ]:
gpr_t_d = t_d + "_gpr"
gpst = prep_for_gpr(training_dvpop_fnames,training_opop_fnames,gpr_t_d)

Now, just for learning, lets sweep over decision variable space but evaluate the GPR emulated objective function value.  You would never do this in practice, but it can be informative:

In [ ]:
gpr_sweep_d = sweep_d+"_gpr"
num_workers = 50
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-ies","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_sweep_d,worker_root='.',
                             port=port)


In [ ]:
sweep_gpr_pe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.par.csv"),index_col=0)
sweep_gpr_oe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.obs.csv"),index_col=0)
gpr_sweep_x = sweep_gpr_pe.loc[:,pst.par_names[0]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_y = sweep_gpr_pe.loc[:,pst.par_names[1]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_z = sweep_gpr_oe.loc[:,pst.obs_names[0]].values.reshape(sweep_steps,sweep_steps)

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(15,3))
get_obj_map(axes[0],sweep_x,sweep_y,sweep_z)
get_obj_map(axes[1],gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
axes[1].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
get_obj_map(axes[2],gpr_sweep_x,gpr_sweep_y,sweep_z-gpr_sweep_z,label="truth minus emulated",levels=[-0.5,0,0.5])
axes[2].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
axes[0].set_title("truth",loc="left")
axes[1].set_title("emulated with training points",loc="left")
axes[2].set_title("difference with training points",loc="left")


OK!  now we can see whats happening - the emulated objective function surface is strongly controlled by the location of the training data points, and, is this case, its not a good representation of the truth surface...yet...

But now lets run pestpp-mou on the GPR emulated model.  This is usually quite fast...

In [ ]:
gpst.control_data.noptmax = 8
gpst.pestpp_options["mou_population_size"] = 20
gpst.write(os.path.join(gpr_t_d,"pest.pst"))
gpr_m_d = gpr_t_d.replace("template","master")
num_workers = 10
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-mou","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_m_d,worker_root='.',
                             port=port)

Now lets plot how the pestpp-mou population evolves across the emulated surface, but also plot it on the true surface just to help us understand what is happening

In [ ]:
gpr_dvpops = [os.path.join(gpr_m_d,f) for f in os.listdir(gpr_m_d) if len(f.split('.')) == 4 and f.endswith("dv_pop.csv") and "archive" not in f]
gpr_dvpops_itr = [int(f.split(".")[1]) for f in gpr_dvpops]
gpr_dvpops = {itr:pd.read_csv(f,index_col=0) for itr,f in zip(gpr_dvpops_itr,gpr_dvpops)}

In [ ]:
for itr in range(max(gpr_dvpops_itr)):
    fig,axes = plt.subplots(1,2,figsize=(10,4))
    ax = axes[0]
    get_obj_map(ax,sweep_x,sweep_y,sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("truth generation {0}".format(itr),loc="left")
    ax = axes[1]
    get_obj_map(ax,gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("emulated generation {0}".format(itr),loc="left")
    plt.show()
    plt.close(fig)

So just what you expected?  Essentially pestpp-mou converged to the minimum of the objective function we gave it, which is the emulated objective function...

Now this where some more cleverness happens:  Lets take that last emulated decision variable population and actually run it thru the complex "model" (which in this case is just the hosaki function...).  This is so that we can "in-fill" our GPR emulator with this new points in decision variable space.  In practice, a lot more cleverness needs to happen to actually decide which points, but for this lil demo, it works...

In [ ]:
gpr_dvpops[max(gpr_dvpops_itr)].to_csv(os.path.join(t_d,"retrain_1_dvpop.csv"))
pst.pestpp_options["mou_dv_population_file"] = "retrain_1_dvpop.csv"
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"pest.pst"))


In [ ]:
m_d += "_1"
num_workers = 10
pyemu.os_utils.start_workers(t_d,"pestpp-mou","pest.pst",
                                 num_workers=num_workers,
                                 master_dir=m_d,worker_root='.',
                                 port=port)

Now load the newly evaluated training points:

In [ ]:
training_dvpop_fname1 = os.path.join(m_d,"pest.0.dv_pop.csv")
training_opop_fname1 = os.path.join(m_d,"pest.0.obs_pop.csv")

And combine these new points with the points we already had (the original few places where we ran the model at the beginning):

In [ ]:
training_dvpop1 = pd.read_csv(training_dvpop_fname1,index_col=0)
training_opop1 = pd.read_csv(training_opop_fname1,index_col=0)
training_dvpop = pd.concat([training_dvpop,training_dvpop1])
training_opop = pd.concat([training_opop,training_opop1])

fig,ax = plt.subplots(1,1,figsize=(6,5))
get_obj_map(ax,sweep_x,sweep_y,sweep_z)
ax.scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
plt.show()
plt.close(fig)

See how we have combined the original points and the new points?

Now lets re-do the GPR training process with this combined decision variable and output/observation population

In [ ]:
training_dvpop_fnames.append(training_dvpop_fname1)
training_opop_fnames.append(training_opop_fname1)
gpr_t_d = t_d + "_gpr1"
gpst = prep_for_gpr(training_dvpop_fnames,training_opop_fnames,gpr_t_d)

Ok, now let's also re-sweep over the emulated surface to see how much better our approximation to the true objective function surface:

In [ ]:
gpr_sweep_d = sweep_d+"_gpr1"
num_workers = 50
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-ies","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_sweep_d,worker_root='.',
                             port=port)

In [ ]:
sweep_gpr_pe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.par.csv"),index_col=0)
sweep_gpr_oe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.obs.csv"),index_col=0)
gpr_sweep_x = sweep_gpr_pe.loc[:,pst.par_names[0]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_y = sweep_gpr_pe.loc[:,pst.par_names[1]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_z = sweep_gpr_oe.loc[:,pst.obs_names[0]].values.reshape(sweep_steps,sweep_steps)
fig, axes = plt.subplots(1,3,figsize=(15,3))
get_obj_map(axes[0],sweep_x,sweep_y,sweep_z)
get_obj_map(axes[1],gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
axes[1].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
get_obj_map(axes[2],gpr_sweep_x,gpr_sweep_y,sweep_z-gpr_sweep_z,label="truth minus emulated",levels=[-0.5,0,0.5])
axes[2].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
axes[0].set_title("truth",loc="left")
axes[1].set_title("emulated with training points",loc="left")
axes[2].set_title("difference with training points",loc="left")

Comparing this to the original emulated surface, we can see it has gotten much better...

Now let's run pestpp-mou on this improved emulated model and visualize how pestpp-mou navigates this new emulated surface:

In [ ]:
gpst.control_data.noptmax = 8
gpst.pestpp_options["mou_population_size"] = 20
gpst.write(os.path.join(gpr_t_d,"pest.pst"))
gpr_m_d = gpr_t_d.replace("template","master")
num_workers = 10
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-mou","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_m_d,worker_root='.',
                             port=port)
gpr_dvpops = [os.path.join(gpr_m_d,f) for f in os.listdir(gpr_m_d) if len(f.split('.')) == 4 and f.endswith("dv_pop.csv") and "archive" not in f]
gpr_dvpops_itr = [int(f.split(".")[1]) for f in gpr_dvpops]

gpr_dvpops = {itr:pd.read_csv(f,index_col=0) for itr,f in zip(gpr_dvpops_itr,gpr_dvpops)}

In [ ]:
for itr in range(max(gpr_dvpops_itr)):
    fig,axes = plt.subplots(1,2,figsize=(10,4))
    ax = axes[0]
    get_obj_map(ax,sweep_x,sweep_y,sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("truth generation {0}".format(itr),loc="left")
    ax = axes[1]
    get_obj_map(ax,gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("emulated generation {0}".format(itr),loc="left")
    plt.show()
    plt.close(fig)

Now lets do that whole process a few times:

In [ ]:
i = 2
gpr_t_d = t_d + "_gpr{0}".format(i-1)
gpr_m_d = gpr_t_d.replace("template","master")
gpr_dvpops = [os.path.join(gpr_m_d,f) for f in os.listdir(gpr_m_d) if len(f.split('.')) == 4 and f.endswith("dv_pop.csv") and "archive" not in f]
gpr_dvpops_itr = [int(f.split(".")[1]) for f in gpr_dvpops]
gpr_dvpops = {itr:pd.read_csv(f,index_col=0) for itr,f in zip(gpr_dvpops_itr,gpr_dvpops)}
gpr_dvpops[max(gpr_dvpops_itr)].to_csv(os.path.join(t_d,"retrain_{0}_dvpop.csv".format(i)))
pst.pestpp_options["mou_dv_population_file"] = "retrain_{0}_dvpop.csv".format(i)
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"pest.pst"))
m_d = "hosaki_model_{0}".format(i)
num_workers = 10
pyemu.os_utils.start_workers(t_d,"pestpp-mou","pest.pst",
                                 num_workers=num_workers,
                                 master_dir=m_d,worker_root='.',
                                 port=port)

training_dvpop_fnamei = os.path.join(m_d,"pest.0.dv_pop.csv")
training_opop_fnamei = os.path.join(m_d,"pest.0.obs_pop.csv")

training_dvpopi = pd.read_csv(training_dvpop_fnamei,index_col=0)
training_opopi = pd.read_csv(training_opop_fnamei,index_col=0)
training_dvpop = pd.concat([training_dvpop,training_dvpopi])
training_opop = pd.concat([training_opop,training_opopi])

fig,ax = plt.subplots(1,1,figsize=(6,5))
get_obj_map(ax,sweep_x,sweep_y,sweep_z)
ax.scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
plt.show()
plt.close(fig)

training_dvpop_fnames.append(training_dvpop_fnamei)
training_opop_fnames.append(training_opop_fnamei)

gpr_t_d = t_d + "_gpr{0}".format(i)
gpr_m_d = gpr_t_d.replace("template","master")
gpst = prep_for_gpr(training_dvpop_fnames,training_opop_fnames,gpr_t_d)
gpr_sweep_d = sweep_d+"_gpr{0}".format(i)
num_workers = 50
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-ies","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_sweep_d,worker_root='.',
                             port=port)

sweep_gpr_pe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.par.csv"),index_col=0)
sweep_gpr_oe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.obs.csv"),index_col=0)
gpr_sweep_x = sweep_gpr_pe.loc[:,pst.par_names[0]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_y = sweep_gpr_pe.loc[:,pst.par_names[1]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_z = sweep_gpr_oe.loc[:,pst.obs_names[0]].values.reshape(sweep_steps,sweep_steps)
fig, axes = plt.subplots(1,3,figsize=(15,3))
get_obj_map(axes[0],sweep_x,sweep_y,sweep_z)
get_obj_map(axes[1],gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
axes[1].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
get_obj_map(axes[2],gpr_sweep_x,gpr_sweep_y,sweep_z-gpr_sweep_z,label="truth minus emulated",levels=[-0.5,0,0.5])
axes[2].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
axes[0].set_title("truth",loc="left")
axes[1].set_title("emulated with training points",loc="left")
axes[2].set_title("difference with training points",loc="left")
plt.show()
plt.close(fig)

gpst.control_data.noptmax = 8
gpst.pestpp_options["mou_population_size"] = 20
gpst.write(os.path.join(gpr_t_d,"pest.pst"))

num_workers = 10
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-mou","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_m_d,worker_root='.',
                             port=port)



for itr in range(max(gpr_dvpops_itr)):
    fig,axes = plt.subplots(1,2,figsize=(10,4))
    ax = axes[0]
    get_obj_map(ax,sweep_x,sweep_y,sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("truth generation {0}".format(itr),loc="left")
    ax = axes[1]
    get_obj_map(ax,gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("emulated generation {0}".format(itr),loc="left")
    plt.show()
    plt.close(fig)

In [ ]:
i = 3
gpr_t_d = t_d + "_gpr{0}".format(i-1)
gpr_m_d = gpr_t_d.replace("template","master")
gpr_dvpops = [os.path.join(gpr_m_d,f) for f in os.listdir(gpr_m_d) if len(f.split('.')) == 4 and f.endswith("dv_pop.csv") and "archive" not in f]
gpr_dvpops_itr = [int(f.split(".")[1]) for f in gpr_dvpops]
gpr_dvpops = {itr:pd.read_csv(f,index_col=0) for itr,f in zip(gpr_dvpops_itr,gpr_dvpops)}
gpr_dvpops[max(gpr_dvpops_itr)].to_csv(os.path.join(t_d,"retrain_{0}_dvpop.csv".format(i)))
pst.pestpp_options["mou_dv_population_file"] = "retrain_{0}_dvpop.csv".format(i)
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"pest.pst"))
m_d = "hosaki_model_{0}".format(i)
num_workers = 10
pyemu.os_utils.start_workers(t_d,"pestpp-mou","pest.pst",
                                 num_workers=num_workers,
                                 master_dir=m_d,worker_root='.',
                                 port=port)

training_dvpop_fnamei = os.path.join(m_d,"pest.0.dv_pop.csv")
training_opop_fnamei = os.path.join(m_d,"pest.0.obs_pop.csv")

training_dvpopi = pd.read_csv(training_dvpop_fnamei,index_col=0)
training_opopi = pd.read_csv(training_opop_fnamei,index_col=0)
training_dvpop = pd.concat([training_dvpop,training_dvpopi])
training_opop = pd.concat([training_opop,training_opopi])

fig,ax = plt.subplots(1,1,figsize=(6,5))
get_obj_map(ax,sweep_x,sweep_y,sweep_z)
ax.scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
plt.show()
plt.close(fig)

training_dvpop_fnames.append(training_dvpop_fnamei)
training_opop_fnames.append(training_opop_fnamei)
gpr_t_d = t_d + "_gpr{0}".format(i)
gpr_m_d = gpr_t_d.replace("template","master")

gpst = prep_for_gpr(training_dvpop_fnames,training_opop_fnames,gpr_t_d)
gpr_sweep_d = sweep_d+"_gpr{0}".format(i)
num_workers = 50
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-ies","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_sweep_d,worker_root='.',
                             port=port)

sweep_gpr_pe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.par.csv"),index_col=0)
sweep_gpr_oe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.obs.csv"),index_col=0)
gpr_sweep_x = sweep_gpr_pe.loc[:,pst.par_names[0]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_y = sweep_gpr_pe.loc[:,pst.par_names[1]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_z = sweep_gpr_oe.loc[:,pst.obs_names[0]].values.reshape(sweep_steps,sweep_steps)
fig, axes = plt.subplots(1,3,figsize=(15,3))
get_obj_map(axes[0],sweep_x,sweep_y,sweep_z)
get_obj_map(axes[1],gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
axes[1].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
get_obj_map(axes[2],gpr_sweep_x,gpr_sweep_y,sweep_z-gpr_sweep_z,label="truth minus emulated",levels=[-0.5,0,0.5])
axes[2].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
axes[0].set_title("truth",loc="left")
axes[1].set_title("emulated with training points",loc="left")
axes[2].set_title("difference with training points",loc="left")
plt.show()
plt.close(fig)

gpst.control_data.noptmax = 8
gpst.pestpp_options["mou_population_size"] = 20
gpst.write(os.path.join(gpr_t_d,"pest.pst"))

num_workers = 10
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-mou","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_m_d,worker_root='.',
                             port=port)



for itr in range(max(gpr_dvpops_itr)):
    fig,axes = plt.subplots(1,2,figsize=(10,4))
    ax = axes[0]
    get_obj_map(ax,sweep_x,sweep_y,sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("truth generation {0}".format(itr),loc="left")
    ax = axes[1]
    get_obj_map(ax,gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("emulated generation {0}".format(itr),loc="left")
    plt.show()
    plt.close(fig)

And one last time...and this time with more emulator generations to polish things off...

In [ ]:
i = 4
gpr_t_d = t_d + "_gpr{0}".format(i-1)
gpr_m_d = gpr_t_d.replace("template","master")
gpr_dvpops = [os.path.join(gpr_m_d,f) for f in os.listdir(gpr_m_d) if len(f.split('.')) == 4 and f.endswith("dv_pop.csv") and "archive" not in f]
gpr_dvpops_itr = [int(f.split(".")[1]) for f in gpr_dvpops]
gpr_dvpops = {itr:pd.read_csv(f,index_col=0) for itr,f in zip(gpr_dvpops_itr,gpr_dvpops)}
gpr_dvpops[max(gpr_dvpops_itr)].to_csv(os.path.join(t_d,"retrain_{0}_dvpop.csv".format(i)))
pst.pestpp_options["mou_dv_population_file"] = "retrain_{0}_dvpop.csv".format(i)
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"pest.pst"))
m_d = "hosaki_model_{0}".format(i)
num_workers = 10
pyemu.os_utils.start_workers(t_d,"pestpp-mou","pest.pst",
                                 num_workers=num_workers,
                                 master_dir=m_d,worker_root='.',
                                 port=port)

training_dvpop_fnamei = os.path.join(m_d,"pest.0.dv_pop.csv")
training_opop_fnamei = os.path.join(m_d,"pest.0.obs_pop.csv")

training_dvpopi = pd.read_csv(training_dvpop_fnamei,index_col=0)
training_opopi = pd.read_csv(training_opop_fnamei,index_col=0)
training_dvpop = pd.concat([training_dvpop,training_dvpopi])
training_opop = pd.concat([training_opop,training_opopi])

fig,ax = plt.subplots(1,1,figsize=(6,5))
get_obj_map(ax,sweep_x,sweep_y,sweep_z)
ax.scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
plt.show()
plt.close(fig)

training_dvpop_fnames.append(training_dvpop_fnamei)
training_opop_fnames.append(training_opop_fnamei)
gpr_t_d = t_d + "_gpr{0}".format(i)
gpr_m_d = gpr_t_d.replace("template","master")
gpst = prep_for_gpr(training_dvpop_fnames,training_opop_fnames,gpr_t_d)
gpr_sweep_d = sweep_d+"_gpr{0}".format(i)
num_workers = 50
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-ies","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_sweep_d,worker_root='.',
                             port=port)

sweep_gpr_pe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.par.csv"),index_col=0)
sweep_gpr_oe = pd.read_csv(os.path.join(gpr_sweep_d,"pest.0.obs.csv"),index_col=0)
gpr_sweep_x = sweep_gpr_pe.loc[:,pst.par_names[0]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_y = sweep_gpr_pe.loc[:,pst.par_names[1]].values.reshape(sweep_steps,sweep_steps)
gpr_sweep_z = sweep_gpr_oe.loc[:,pst.obs_names[0]].values.reshape(sweep_steps,sweep_steps)
fig, axes = plt.subplots(1,3,figsize=(15,3))
get_obj_map(axes[0],sweep_x,sweep_y,sweep_z)
get_obj_map(axes[1],gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
axes[1].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
get_obj_map(axes[2],gpr_sweep_x,gpr_sweep_y,sweep_z-gpr_sweep_z,label="truth minus emulated",levels=[-0.5,0,0.5])
axes[2].scatter(training_dvpop.loc[:,pst.par_names[0]],training_dvpop.loc[:,pst.par_names[1]],marker='^',c='w',s=20)
axes[0].set_title("truth",loc="left")
axes[1].set_title("emulated with training points",loc="left")
axes[2].set_title("difference with training points",loc="left")
plt.show()
plt.close(fig)

gpst.control_data.noptmax = 15
gpst.pestpp_options["mou_population_size"] = 20
gpst.write(os.path.join(gpr_t_d,"pest.pst"))

num_workers = 10
pyemu.os_utils.start_workers(gpr_t_d,"pestpp-mou","pest.pst",
                             num_workers=num_workers,
                             master_dir=gpr_m_d,worker_root='.',
                             port=port)



for itr in range(max(gpr_dvpops_itr)):
    fig,axes = plt.subplots(1,2,figsize=(10,4))
    ax = axes[0]
    get_obj_map(ax,sweep_x,sweep_y,sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("truth generation {0}".format(itr),loc="left")
    ax = axes[1]
    get_obj_map(ax,gpr_sweep_x,gpr_sweep_y,gpr_sweep_z)
    ax.scatter(gpr_dvpops[itr].loc[:,gpst.par_names[0]],gpr_dvpops[itr].loc[:,gpst.par_names[1]],marker='.',c='w',s=10)
    ax.set_title("emulated generation {0}".format(itr),loc="left")
    plt.show()
    plt.close(fig)

There you have it - after evaluating only a few populations with the complex process-based model (here that was only the hosaki function), we can effectively optimize a function with the otherwise every expensive global solvers in pestpp-mou.